In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [3]:
df['Launch Site'].value_counts()

CCAFS LC-40     26
KSC LC-39A      13
VAFB SLC-4E     10
CCAFS SLC-40     7
Name: Launch Site, dtype: int64

In [4]:
from dash import Dash, html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [5]:
min_payload=df['Payload Mass (kg)'].min()
max_payload=df['Payload Mass (kg)'].max()

In [9]:
app=Dash(__name__)

In [10]:
app.layout=html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                      style={'textAlign':'center', 'color':'#503D36',
                                             'font-size':40}),
                              dcc.Dropdown(id='site-dropdown',
                                           options=[{'label':'All Sites', 'value':'ALL'},
                                                    {'label':'CCAFS LC-40', 'value':'CCAFS LC-40'},
                                                    {'label':'KSC LC-39A', 'value':'KSC LC-39A'},
                                                    {'label':'VAFB SLC-4E', 'value':'VAFB SLC-4E'},
                                                    {'label':'CCAFS SLC-40', 'value':'CCAFS SLC-40'},
                                                   ],
                                           placeholder='Select a Launch Site here',
                                           value='ALL',
                                           searchable=True
                                          ),
                              html.Br(),
                              html.Div(dcc.Graph(id='success-pie-chart')),
                              html.Br(),
                              html.P("Payload range (Kg):"),
                              
                              dcc.RangeSlider(id='payload-slider',
                                              min=0, max=10000, step=1000,
                                              marks={0:'0',
                                                     2500:'2500',
                                                     5000:'5000',
                                                     7500:'7500',
                                                     10000:'10000'},
                                              value=[min_payload, max_payload]),
                              html.Div(dcc.Graph(id='success-payload-scatter-chart'))
                             ])

@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
             Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    if(entered_site=='ALL'):
        fig=px.pie(data_frame=df, 
                   values='class', 
                   names='Launch Site',
                   title='Total Success launches by Site')
        return fig
    else:
        df_t=df.loc[df['Launch Site']==entered_site]
        fig=px.pie(data_frame=df_t, names='class', title='Total Launch for specific Site')
        return fig
    
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')])

def update_graph(entered_site, payload_slider):
    if(entered_site=='ALL'):
        df_filter=df[(df['Payload Mass (kg)'] >=payload_slider[0]) & (df['Payload Mass (kg)'] <=payload_slider[1])]
        scatter=px.scatter(data_frame=df_filter,
                               x='Payload Mass (kg)',
                               y='class',
                               color='Booster Version Category')
        return scatter
    else:
        df_t=df.loc[df['Launch Site'] == entered_site]
        df_filter=df_t[(df_t['Payload Mass (kg)'] >=payload_slider[0]) & (df_t['Payload Mass (kg)'] <=payload_slider[1])]
        scatter=px.scatter(data_frame=df_filter,
                               x='Payload Mass (kg)',
                               y='class',
                               color='Booster Version')
        return scatter

if __name__=='__main__':
    app.run_server(port=8000)